In [1]:
from utils import PluginFeatureExtractor
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell

extractor = PluginFeatureExtractor(midi_note=24, note_length_secs=0.8,
                                   render_length_secs=1.8,
                                   pickle_path="normalisers/",
                                   warning_mode="ignore")

path = "/home/tollie/Development/vsts/dexed/Builds/Linux/build/Dexed.so"
extractor.load_plugin(path)

if extractor.need_to_fit_normalisers():
    extractor.fit_normalisers(10000)

(features, parameters) = extractor.get_random_normalised_example()

learning_rate = 0.001
training_iters = 2000000
batch_size = 128
display_step = 10
save_step = 50
number_hidden = 128
number_layers = 4
number_input = int(features.shape[1])
number_timesteps = int(features.shape[0])
number_outputs = len(parameters)

x = tf.placeholder("float", [None, number_timesteps, number_input], name="Features")
y = tf.placeholder("float", [None, number_outputs], name="Synth_Patch")

# Create model
def RNN(x, weights, biases):
    # Prepare data shape to match rnn function requirements.
    # Current data input shape: (batch_size, number_timesteps, number_input)
    # Required shape: (number_timesteps, batch_size, number_input)
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, number_input])
    x = tf.split(0, number_timesteps, x)

    # embeddings = tf.get_variable('embedding_matrix', [number_outputs, number_hidden])
    # lstm_inputs = tf.nn.embedding_lookup(embeddings, x)

    lstm_cell = rnn_cell.LSTMCell(number_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cell = rnn_cell.MultiRNNCell([lstm_cell] * number_layers, state_is_tuple=True)
    init_state = lstm_cell.zero_state(batch_size, tf.float32)
    outputs, states = rnn.rnn(cell=lstm_cell, inputs=x, dtype=tf.float32, initial_state=init_state)

    # Linear activation using rnn inner loop last output
    return tf.add(tf.matmul(outputs[-1], weights['out']), biases['out'])

def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.01), name=name)

# Store layers weight & bias
weights = {
    'out': init_weights([number_hidden, number_outputs], "weights_out")
}
biases = {
    'out': init_weights([number_outputs], "biases_out")
}

tf.summary.histogram("out", weights['out'])

# Construct model
prediction = RNN(x, weights, biases)

# Add ops to save and restore all variables.
saver = tf.train.Saver()

# Launching the graph.
with tf.Session() as sess:
    
    # Restore variables from disk.
    saver.restore(sess, "models/deep_rnn_model.ckpt")
    print("Model restored.")

    test_batch = []
    for i in range(batch_size):
        test_batch += [extractor.get_random_normalised_example()]

    split_test = map(list, zip(*test_batch))
    batch_x = split_test[0]
    batch_y = split_test[1]

    pred = sess.run([prediction], feed_dict={ x: batch_x,
                                              prob_keep_input: 1.0,
                                              prob_keep_hidden: 1.0 })

    all_tests = []

    for i in range(len(pred[0])):
        total_abs = 0

        for param in range(len(pred[0][0])):
            total_abs += abs(pred[0][i][param] - batch_y[i][param])

        absolute_distance_table = " " +("%04d" % i) + " : " + ("%.5f" % round(total_abs, 5))
        all_tests.append((total_abs, absolute_distance_table, pred[0][i], batch_y[i]))

    all_tests.sort(key=lambda x: x[0])

    print "\n\n      In order of most to least similar predictions compared to the actual parameters:\n\n      Index   Total abs"
    print "       _______________"
    for i in range(len(all_tests)):
        print ("%04d" % i) + ") " + all_tests[i][1]

    for i in range(1):
        patch_predicted = add_patch_indices(all_tests[i][2])
        patch_actual = add_patch_indices(all_tests[i][3])

        extractor.set_patch(patch_predicted)
        file_name = str(i) + "_pred.wav"
        pred_audio = engine.get_audio_frames()
        engine.set_patch(patch_actual)
        file_name = str(i) + "_actual.wav"
        actual_audio = engine.get_audio_frames()

/usr/local/lib/python2.7/dist-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Successfully loaded plugin.


NotFoundError: Key RNN/MultiRNNCell/Cell3/LSTMCell/B not found in checkpoint
	 [[Node: save/RestoreV2_6 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/RestoreV2_6/tensor_names, save/RestoreV2_6/shape_and_slices)]]
	 [[Node: save/RestoreV2_4/_5 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_37_save/RestoreV2_4", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'save/RestoreV2_6', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/t